[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/stefmolin/pandas-workshop/main?urlpath=lab/tree/notebooks/2-data_wrangling.ipynb) [![View slides in browser](https://img.shields.io/badge/view-slides-orange?logo=reveal.js&logoColor=white)](https://stefaniemolin.com/pandas-workshop/#/section-2)

---



# Lecture 6: Data Cleaning and Preprocessing

To prepare our data for analysis, we need to perform data wrangling. In this section, we will learn how to clean and reformat data (e.g., renaming columns and fixing data type mismatches), restructure/reshape it, and enrich it (e.g., discretizing columns, calculating aggregations, and combining data sources).

## Data cleaning with Pandas

In this section, we will take a look at creating, renaming, and dropping columns; type conversion; and sorting &ndash; all of which make our analysis easier. We will be working with the Messy HR data.

In [70]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('../Messy-dataset/messy_HR_data.csv')

df.head()

,Name,Age,Salary,Gender,Department,Position,Joining Date,Performance Score,Email,Phone Number
0,grace,25,50000,Male,HR,Manager,"April 5, 2018",D,email@example.com,NaN
1,david,NaN,65000,Female,Finance,Director,2020/02/20,F,user@domain.com,123-456-7890
2,hannah,35,SIXTY THOUSAND,Female,Sales,Director,01/15/2020,C,email@example.com,098-765-4321
3,eve,NaN,50000,Female,IT,Manager,"April 5, 2018",A,name@company.org,
4,grace,NaN,NAN,Female,Finance,Manager,01/15/2020,F,name@company.org,098-765-4321


*Source: [Messy-dataset](https://github.com/eyowhite/Messy-dataset).*

### 1. Inspect the Data

In [71]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               1000 non-null   object
 1   Age                841 non-null    object
 2   Salary             1000 non-null   object
 3   Gender             1000 non-null   object
 4   Department         1000 non-null   object
 5   Position           1000 non-null   object
 6   Joining Date       1000 non-null   object
 7   Performance Score  1000 non-null   object
 8   Email              610 non-null    object
 9   Phone Number       815 non-null    object
dtypes: object(10)
memory usage: 78.3+ KB


,Name,Age,Salary,Gender,Department,Position,Joining Date,Performance Score,Email,Phone Number
count,1000,841,1000,1000,1000,1000,1000,1000,610,815
unique,10,5,6,3,5,5,5,5,3,4
top,alice,thirty,65000,Male,Finance,Assistant,2020/02/20,B,user@domain.com,123-456-7890
freq,118,176,184,355,218,214,232,225,213,236


What to look for:
- Missing values
- Wrong data types (dates as strings, numbers as objects)
- Unexpected zeros or negatives
- Outliers

### 2. Fixed data types

In [72]:
df = df.copy()

df = df.applymap(
    lambda x: x.strip() if isinstance(x, str) else x
)

df.replace(
    ['N/A', 'NA', 'na', 'None', '', '?', 'unknown'],
    pd.NA,
    inplace=True
)

In [73]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce').astype('Int64')

In [74]:
df['Salary'] = (
    df['Salary']
    .str.replace(r'[\$,]', '', regex=True)
)

df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')


In [75]:
df['Joining Date'] = pd.to_datetime(
    df['Joining Date'],
    errors='coerce'
)

In [76]:
categorical_cols = [
    'Gender',
    'Department',
    'Position',
    'Performance Score'
]

df[categorical_cols] = df[categorical_cols].astype('category')

In [77]:
string_cols = [
    'Name',
    'Email',
    'Phone Number'
]

df[string_cols] = df[string_cols].astype('string')

In [78]:
df.loc[~df['Email'].str.contains('@', na=False), 'Email'] = pd.NA
df['Phone Number'] = df['Phone Number'].str.replace(r'\D', '', regex=True)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               1000 non-null   string        
 1   Age                665 non-null    Int64         
 2   Salary             690 non-null    float64       
 3   Gender             1000 non-null   category      
 4   Department         1000 non-null   category      
 5   Position           1000 non-null   category      
 6   Joining Date       186 non-null    datetime64[ns]
 7   Performance Score  1000 non-null   category      
 8   Email              610 non-null    string        
 9   Phone Number       624 non-null    string        
dtypes: Int64(1), category(4), datetime64[ns](1), float64(1), string(3)
memory usage: 52.6 KB


### 3. Handle Missing Values

In [58]:
df.isna().sum().sort_values(ascending=False)

Joining Date         814
Email                390
Phone Number         376
Age                  335
Salary               310
Name                   0
Gender                 0
Department             0
Position               0
Performance Score      0
dtype: int64

In [81]:
df = df.dropna(subset=['Name'])
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Salary'] = df['Salary'].fillna(df['Salary'].median())
df['Gender'] = df['Gender'].fillna(
    df['Gender'].mode()[0]
)
df['Department'] = df['Department'].fillna(
    df['Department'].mode()[0]
)
df['Position'] = df['Position'].fillna(
    df['Position'].mode()[0]
)

# Keep the same as original
df['Performance Score'] = df['Performance Score']
df['Joining Date'] = df['Joining Date']
df[['Email', 'Phone Number']] = df[['Email', 'Phone Number']]

df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               1000 non-null   string        
 1   Age                1000 non-null   Int64         
 2   Salary             1000 non-null   float64       
 3   Gender             1000 non-null   category      
 4   Department         1000 non-null   category      
 5   Position           1000 non-null   category      
 6   Joining Date       186 non-null    datetime64[ns]
 7   Performance Score  1000 non-null   category      
 8   Email              610 non-null    string        
 9   Phone Number       624 non-null    string        
dtypes: Int64(1), category(4), datetime64[ns](1), float64(1), string(3)
memory usage: 52.6 KB


In [82]:
df.isna().sum().sort_values(ascending=False)

Joining Date         814
Email                390
Phone Number         376
Name                   0
Age                    0
Salary                 0
Gender                 0
Department             0
Position               0
Performance Score      0
dtype: int64

### 4. Remove Duplicates

In [83]:
df.duplicated().sum()

np.int64(0)

### 5. Validate Data Ranges & Logic

In [86]:
df = df[(df['Age'] >= 16) & (df['Age'] <= 75)]
df = df[df['Salary'] > 0]

### 6. Detect & Handle Outliers

In [87]:
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

df = df[
    (df['Salary'] >= Q1 - 1.5 * IQR) &
    (df['Salary'] <= Q3 + 1.5 * IQR)
]

### 7. Standardize Text Data

In [88]:
text_cols = ['Gender', 'Department', 'Position']

df[text_cols] = (
    df[text_cols]
    .apply(lambda col: col.str.lower().str.strip())
)

## Data cleaning with PandasAI using LLMs

In [26]:
# safe for JSON serialization.
# df = df.copy()
# df = df.applymap(
#     lambda x: x.strip() if isinstance(x, str) else x
# )
# df.replace(
#     ['N/A', 'NA', 'na', 'None', '', '?', 'unknown'],
#     pd.NA,
#     inplace=True
# )
# df['Age'] = pd.to_numeric(df['Age'], errors='coerce').astype('Int64')
# df['Salary'] = (
#     df['Salary']
#     .str.replace(r'[\$,]', '', regex=True)
# )

# df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
# df['Joining Date'] = pd.to_datetime(
#     df['Joining Date'],
#     errors='coerce'
# )
# categorical_cols = [
#     'Gender',
#     'Department',
#     'Position',
#     'Performance Score'
# ]

# df[categorical_cols] = df[categorical_cols].astype('category')

# # Convert categories to strings
# cat_cols = df.select_dtypes(['category']).columns
# df[cat_cols] = df[cat_cols].astype(str)
# # Convert datetime to ISO format strings
# date_cols = df.select_dtypes(['datetime64[ns]']).columns
# df[date_cols] = df[date_cols].astype(str)

# string_cols = [
#     'Name',
#     'Email',
#     'Phone Number'
# ]

# df[string_cols] = df[string_cols].astype('string')

In [35]:
import warnings
warnings.filterwarnings("ignore")

### 1. Setup PandasAI with a LLM

In [23]:
import os
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm.openai import OpenAI

# Set your OpenAI API key
# os.environ["OPENAI_API_KEY"] = "sk-your_api_key_here"  # replace with your key

# Load your HR dataset
# df = pd.read_csv("../Messy-dataset/messy_HR_data.csv")
df = pd.read_csv("../Messy-dataset/cleaned_messy_HR_data.csv")

# Initialize LLM and SmartDataframe
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = OpenAI(api_token=openai_api_key)
sdf = SmartDataframe(df.sample(50), config={"llm": llm})

### 2. Prompt: 
#### 2.1 Summary & Missing Data

In [37]:
import sys
sys.setrecursionlimit(5000)
# Ask AI to summarize the dataset
summary = sdf.chat("Summarize this HR dataset: column types, missing values, and basic stats") # 
print(summary)

                        column_types  missing_values  \
Name                          string               0   
Age                            Int64               0   
Salary                       float64               0   
Gender                        object               0   
Department                    object               0   
Position                      object               0   
Joining Date          datetime64[ns]               0   
Performance Score            float64              50   
Email                         string              22   
Phone Number                  string              21   
Tenure                         int64               0   
Salary_Band                 category               0   
Performance_Category        category              50   

                                                            basic_stats  
Name                  {'count': 50, 'unique': 9, 'top': 'alice', 'fr...  
Age                   {'count': 50.0, 'unique': <NA>, 'top': <NA>, 

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/pipelines/chat/code_execution.py", line 133, in execute
    {"content_type": "response", "value": ResponseSerializer.serialize(result)},
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandasai/responses/response_serializer.py", line 27, in serialize
    df_dict = ResponseSerializer.serialize_dataframe(result["value"])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### 2.2 Detect Outliers

In [38]:
outliers = sdf.chat("Detect outliers in Age, Salary, and Performance Score columns and suggest handling")
print(outliers)

Age Outliers:
        Name  Age   Salary  Gender Department   Position Joining Date  \
10     david   25  70000.0    Male  Marketing    Analyst   2019-12-01   
417     jack   50  50000.0    Male         IT    Analyst   2018-04-05   
793  charlie   50  70000.0  Female    Finance    Manager   2019-03-25   
181      eve   50  70000.0   Other  Marketing      Clerk   2018-04-05   
704      bob   25  70000.0  Female  Marketing  Assistant   2019-12-01   
635   hannah   25  55000.0    Male         HR   Director   2018-04-05   
743  charlie   25  70000.0  Female         IT    Manager   2019-03-25   
122  charlie   25  65000.0  Female      Sales    Manager   2020-01-15   
235      eve   25  65000.0   Other         HR    Manager   2018-04-05   
831    alice   50  65000.0    Male         IT    Manager   2019-12-01   
502    alice   25  50000.0  Female      Sales    Analyst   2020-01-15   
883   hannah   50  65000.0  Female  Marketing  Assistant   2020-02-20   
647      ivy   25  70000.0   Other   

#### 2.3 Automated Cleaning Suggestions

In [29]:
cleaned_df = sdf.chat("""
Clean the dataset:
- Impute missing Age with median
- Impute missing Salary with median
- Fill missing Gender/Department/Position with mode
- Leave Email and Phone Number missing
- Remove duplicates
Return the cleaned DataFrame
""")

In [30]:
cleaned_df

,Name,Age,Salary,Gender,Department,Position,Joining Date,Performance Score,Email,Phone Number
10,david,25,70000.0,Male,Marketing,Analyst,2019-12-01,B,<NA>,123-456-7890
576,alice,40,65000.0,Male,Finance,Clerk,2018-04-05,F,<NA>,<NA>
232,bob,40,65000.0,Female,Finance,Analyst,2020-01-15,D,email@example.com,098-765-4321
611,hannah,40,55000.0,Other,Sales,Manager,2019-12-01,A,email@example.com,123-456-7890
417,jack,50,50000.0,Male,IT,Analyst,2018-04-05,A,<NA>,<NA>
793,charlie,50,70000.0,Female,Finance,Manager,2019-03-25,B,user@domain.com,<NA>
965,david,35,65000.0,Female,Marketing,Director,2018-04-05,A,email@example.com,123-456-7890
181,eve,50,70000.0,Other,Marketing,Clerk,2018-04-05,B,name@company.org,<NA>
783,jack,40,50000.0,Male,IT,Assistant,2018-04-05,B,<NA>,<NA>
704,bob,25,70000.0,Female,Marketing,Assistant,2019-12-01,C,<NA>,<NA>


#### Exploratory Data Analysis (EDA)

In [31]:
eda_summary = sdf.chat("""
Perform EDA:
- Show average Salary by Department and Position
- Show distribution of Age and Salary
- Show performance scores by Gender
- Visualize Salary distribution by Department
""")
print(eda_summary)

   Department   Position        Salary
0     Finance    Analyst  61666.666667
1     Finance  Assistant  70000.000000
2     Finance      Clerk  60000.000000
3     Finance    Manager  55000.000000
4          HR    Analyst  55000.000000
5          HR  Assistant  70000.000000
6          HR      Clerk  60000.000000
7          HR   Director  60000.000000
8          HR    Manager  67500.000000
9          IT    Analyst  52500.000000
10         IT  Assistant  60000.000000
11         IT   Director  70000.000000
12         IT    Manager  67500.000000
13  Marketing    Analyst  68333.333333
14  Marketing  Assistant  67500.000000
15  Marketing      Clerk  62500.000000
16  Marketing   Director  61000.000000
17  Marketing    Manager  50000.000000
18      Sales    Analyst  50000.000000
19      Sales  Assistant  60000.000000
20      Sales      Clerk  63333.333333
21      Sales   Director  65000.000000
22      Sales    Manager  60000.000000


<Figure size 1000x600 with 0 Axes>

Text summary (tables, means, medians)
Generates plots inline (histograms, boxplots, bar charts)

#### Feature Engineering

In [39]:
features = sdf.chat("""
Suggest new features for predicting Performance Score:
- Derive Tenure from Joining Date
- Create Salary_Band based on quartiles
- Suggest any other useful columns
""")
print(features)

        Name  Age   Salary  Gender Department   Position Joining Date  \
10     david   25  70000.0    Male  Marketing    Analyst   2019-12-01   
576    alice   40  65000.0    Male    Finance      Clerk   2018-04-05   
232      bob   40  65000.0  Female    Finance    Analyst   2020-01-15   
611   hannah   40  55000.0   Other      Sales    Manager   2019-12-01   
417     jack   50  50000.0    Male         IT    Analyst   2018-04-05   
793  charlie   50  70000.0  Female    Finance    Manager   2019-03-25   
965    david   35  65000.0  Female  Marketing   Director   2018-04-05   
181      eve   50  70000.0   Other  Marketing      Clerk   2018-04-05   
783     jack   40  50000.0    Male         IT  Assistant   2018-04-05   
704      bob   25  70000.0  Female  Marketing  Assistant   2019-12-01   
236    grace   40  55000.0    Male      Sales      Clerk   2020-01-15   
294    alice   40  50000.0   Other  Marketing    Manager   2019-12-01   
635   hannah   25  55000.0    Male         HR   Dir

#### Validation / QA

In [40]:
validation = sdf.chat("""
Check the dataset for any remaining issues:
- Missing values
- Duplicates
- Logical inconsistencies (Age < 16 or > 75, Salary < 0)
""")
print(validation)

{'type': 'string', 'value': 'Missing Values:\nName                     0\nAge                      0\nSalary                   0\nGender                   0\nDepartment               0\nPosition                 0\nJoining Date             0\nPerformance Score       50\nEmail                   22\nPhone Number            21\nTenure                   0\nSalary_Band              0\nPerformance_Category    50\nAge_Group                0\ndtype: int64\n\nDuplicates: 0\n\nAge Inconsistencies:\nEmpty DataFrame\nColumns: [Name, Age, Salary, Gender, Department, Position, Joining Date, Performance Score, Email, Phone Number, Tenure, Salary_Band, Performance_Category, Age_Group]\nIndex: []\n\nSalary Inconsistencies:\nEmpty DataFrame\nColumns: [Name, Age, Salary, Gender, Department, Position, Joining Date, Performance Score, Email, Phone Number, Tenure, Salary_Band, Performance_Category, Age_Group]\nIndex: []'}
Missing Values:
Name                     0
Age                      0
Salary           

AI highlights issues

Suggests corrective actions if needed